In [1]:
from common.tools import Reddit
from common.translations import IO
import pandas as pd
from datetime import datetime as dt
import os
import matplotlib.pyplot as plt

In [2]:
r = Reddit()
r.subreddits

['SecurityAnalysis', 'StockMarket', 'wallstreetbets', 'wallstreetbets.pd']

In [3]:
subreddit = r.subreddits[2]

In [4]:
filepath = IO["OUTPUT"]["THREADS"] + subreddit + ".pd"
df = pd.read_pickle(filepath)

In [5]:
df.head()

,created_at,title,selftext,score,permalink,all_comments
0,2021-05-16,Life savings of 43k into UWMC - I have a crapp...,,1120.0,/r/wallstreetbets/comments/ndgwki/life_savings...,[Can't you just turn your civic into a sports ...
1,2021-06-08,"My current $3.7 mil portfolio, powered entirel...",,4467.0,/r/wallstreetbets/comments/nv5gs3/my_current_3...,[You could retire comfortably. I’m glad you’ve...
2,2021-06-09,$UWMC This rocket is fueled and ready 🚀🚀🚀,,358.0,/r/wallstreetbets/comments/nw73p0/uwmc_this_ro...,[I’m in for 400 shares at 7.90 cost basis. Bee...
3,2021-06-10,$ASO - Your YOLOs don't have to be retarded(A ...,All you retards dumping your life savings in c...,208.0,/r/wallstreetbets/comments/nwysob/aso_your_yol...,"[Smooth brain enjoy, picturebook good, easy.\n..."
4,2021-06-10,"$CLOV was a meme stock, but it was undervalued...","I’ll admit it, I didn’t do any fucking researc...",240.0,/r/wallstreetbets/comments/nwz20e/clov_was_a_m...,[Just a dumb ape here but isn't Ortex reportin...


# Text pre-processing

In [6]:
import re
import string

In [7]:
#df["text"] = df.apply(lambda row: row["title"] + " ".join(row["all_comments"]), axis=1)
df["text"] = df.apply(lambda row: row["title"] + row["selftext"], axis=1)

In [8]:
def cleaning(text):
    text = text.lower()
    text = re.sub("\[.*\]", "", text)
    text = re.sub("\(.*\)", "", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub("\w*\d\w*", "", text)
    text = re.sub("\s{2}", " ", text)
    text = text.encode("ascii", "ignore").decode("ascii") # removing emojis
    return text

In [9]:
df["text"] = df.text.apply(lambda text: cleaning(text))
#df["title"] = df.title.apply(lambda title: cleaning(title))
#df["selftext"] = df.selftext.apply(lambda text: cleaning(text))
#df["all_comments"] = df.all_comments.apply(lambda comments: [cleaning(comment) for comment in comments])

In [10]:
import nltk
nltk.download('wordnet')
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [11]:
def get_lemma(word):
    lemma = wordnet.morphy(word)
    if lemma:
        return lemma
    return word

In [12]:
def get_root_word(word):
    return WordNetLemmatizer().lemmatize(word)

In [13]:
stop_words = set(stopwords.words("english"))

In [14]:
df["tokens"] = df.text.apply(lambda t: [get_lemma(token) for token in word_tokenize(t) if token not in stop_words])

# LDA

In [15]:
import gensim
from gensim import corpora

C:\Users\Vincent\AppData\Roaming\Python\Python37\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [16]:
id2word = corpora.Dictionary(df.tokens)

In [18]:
corpus = [id2word.doc2bow(tokens) for tokens in df.tokens]

In [68]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [69]:
topics = lda_model.print_topics(num_words=10)

In [70]:
for topic in topics:
    print(topic)

(0, '0.026*"company" + 0.024*"share" + 0.015*"growth" + 0.011*"revenue" + 0.010*"million" + 0.010*"market" + 0.009*"data" + 0.006*"billion" + 0.006*"cash" + 0.006*"available"')
(1, '0.020*"levi" + 0.019*"loan" + 0.014*"increase" + 0.010*"price" + 0.010*"earnings" + 0.010*"year" + 0.009*"demand" + 0.007*"sales" + 0.007*"home" + 0.007*"industry"')
(2, '0.018*"product" + 0.018*"company" + 0.016*"ttcf" + 0.016*"tilray" + 0.012*"yolo" + 0.011*"wish" + 0.011*"cannabis" + 0.010*"food" + 0.009*"merger" + 0.008*"customer"')
(3, '0.014*"stock" + 0.009*"price" + 0.009*"short" + 0.009*"sofi" + 0.008*"like" + 0.008*"share" + 0.007*"get" + 0.006*"see" + 0.006*"market" + 0.006*"call"')
(4, '0.011*"car" + 0.008*"us" + 0.008*"ev" + 0.008*"vehicle" + 0.006*"spac" + 0.006*"cell" + 0.006*"state" + 0.006*"canoo" + 0.005*"technology" + 0.005*"design"')


# Scores

In [71]:
import pyLDAvis
import pyLDAvis.gensim_models

In [72]:
from gensim.models import CoherenceModel

In [73]:
print("Perplexity:", lda_model.log_perplexity(corpus))

Perplexity: -8.118002870761714


In [74]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=df.tokens, dictionary=id2word, coherence="c_v")
print("Coherence Score:", coherence_model_lda.get_coherence())

Coherence Score: 0.39798589040340004


In [75]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\Vincent\AppData\Roaming\Python\Python37\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.267822 -0.057776       1        1  58.087255
0     -0.053597 -0.016181       2        1  15.229785
1     -0.040099  0.107871       3        1  10.479117
2      0.194556 -0.230240       4        1   8.844605
4      0.166962  0.196326       5        1   7.359238, topic_info=           Term         Freq        Total Category  logprob  loglift
283       share  1474.000000  1474.000000  Default  30.0000  30.0000
48      company  1657.000000  1657.000000  Default  29.0000  29.0000
13844      levi   377.000000   377.000000  Default  28.0000  28.0000
1511       loan   371.000000   371.000000  Default  27.0000  27.0000
96     increase   559.000000   559.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
877         use    58.415718   401.872059   Topic5  -5.4383   0.6807
1634   facility    40.743767    55.358067   Topic5  -5.7985   2.3027
467     include    47.649822   211.408176   Topic5  -5.6420   1.1193
674        plan    44.519746   167.006415   Topic5  -5.7099   1.2871
48      company    49.336235  1657.893978   Topic5  -5.6072  -0.9054

[307 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
25        1  0.569372    account
25        4  0.428559    account
1973      2  0.788562    acquire
1973      5  0.204442    acquire
2290      2  0.987491  aggregate
...     ...       ...        ...
177       3  0.320144       year
177       5  0.003318       year
581       1  0.100034       yolo
581       4  0.900303       yolo
179       2  0.997911        yoy

[470 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3, 5])